In [1]:
from dotenv import load_dotenv
import os
import openai
import time
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
import warnings

load_dotenv()
warnings.filterwarnings("ignore")

In [2]:
DATA_PATH = "data/updated_training_data.jsonl"


def train_model():
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # Upload file
    print("Uploading file to OpenAI...")
    upload_response = client.files.create(
        file=open(DATA_PATH, "rb"), purpose="fine-tune"
    )
    uploaded_file_id = upload_response.id
    print("File uploaded. File ID:", uploaded_file_id)

    # Create fine-tune job
    print("Creating fine-tune job...")
    fine_tune_response = client.fine_tuning.jobs.create(
        training_file=uploaded_file_id,
        model="gpt-4o-2024-08-06",
        hyperparameters={
            "batch_size": 2,
            "learning_rate_multiplier": 0.05,
            "n_epochs": 16,
        },
    )
    fine_tune_id = fine_tune_response.id
    print("Fine-tune job created. Response:", fine_tune_response)

    # Wait for completion
    print("Waiting for fine-tune job to complete... This can take a while.")
    status = None
    while status not in ["succeeded", "failed"]:
        time.sleep(30)  # wait before checking
        job_status = client.fine_tuning.jobs.retrieve(fine_tune_id)
        status = job_status.status
        print(f"Current status: {status}")

    print("Fine-tune job finished with status:", status)

    # Handle completion
    if status == "succeeded":
        fine_tuned_model = job_status.fine_tuned_model
        print("Fine-tuned model name:", fine_tuned_model)
        with open("fine_tuned_model_name.txt", "w") as f:
            f.write(fine_tuned_model)
    # elif status == "failed":
    #     print("Fine-tuning failed. Retrieving failure reasons...")
    #     failure_details = job_status.get("failure_details", "No detailed information provided.")
    #     print("Failure details:", failure_details)

In [3]:
SCHEMA_PROMPT = """
  You are an expert in generating JSON schemas for the AdvancedFormNode component. Your output must exactly match the following structure:

  {{
    "topOptions": [
      {{
        "id": "<unique_id>",
        "name": "<field_name>",
        "label": "<field_label>",
        "type": "<field_type>",
        "default_value": "<default_value>/null",
        "is_read_only": true/false,
        "required": true/false,
        "options": [
          {{ "display_value": "<label>", "value": "<value>" }}
        ],
        "options_query": "query|<query_to_fetch_options>/null",
        "onchange": "<onchange_behavior>/null",
        "display": "<none/"">",
        "order": <integer>,
        "size": <integer>
      }}
    ],
    "listOptions": [
      {{
        "id": "<unique_id>",
        "name": "<field_name>",
        "label": "<field_label>",
        "type": "<field_type>",
        "default_value": "<default_value>",
        "required": true/false,
        "is_read_only": true/false,
        "options": [
          {{ "display_value": "<label>", "value": "<value>" }}
        ],
        "options_query": "query|<query_to_fetch_options>/null",
        "display": "<none/"">", if for any field display is none then and then only add rows: [] otherwise there should be no rows key in the object for display: ""
        "rows": [],
        "order": <integer>,
        "size": <integer>
      }}
    ],
    "defaultColumns": [
      {{
        "dataIndex": "<column_name>",
        "title": "<column_label>",
        "type": "<column_type>",
        "editable": true/false,
        "options": [
          {{ "display_value": "<label>", "value": "<value>" }}
        ],
        "options_query": "query|<query_to_fetch_options>",
      }}
    ],
    "dependencies": [
      {{
        "type": "<dependency_type>",
        "dependentFields": [
          {{
            "fieldType": "<field_type>",
            "fieldDataIndex": "<field_name>"
          }}
        ],
        "targetFields": [
          {{
            "fieldType": "<field_type>",
            "fieldDataIndex": "<field_name>",
            "function": "<javascript_function> description of function structure provided below for each dependency type."
          }}
        ]
      }}
    ]
  }}

  Dependency Key Descriptions and Function Requirements

  Each dependency type has specific keys and nuances. Below are the detailed descriptions of the keys and requirements for the function field in targetFields:

  1. Transformation Dependency
  - type: "transformation"
  - description: This dependency has capacity to directly update the form or form list or table element. Thus if any requirement comes which deals with updating the value of the field instead of modifying the input schema dependency type transformation is the go to place. Whenever the user modifies any of the field from the form or form list or the table present inside the dependentFields function inside the targetFields is executed. the function in targetFields returns the value for the fieldDataIndex for the particular targetFields object. whenever any one of the dependentFields is updated by user for a particular dependency all the objects inside the targetFields will be executed one by one. One thing to note here for you is if targetFields contains 3 objects then the output from the first object is input for the second and output of second is input for the third. So net net there can be multiple dependentFields, multiple targetFields, dependency will be executed if user changes one of the dependentFields, once called all the targetFields objects will be processed. and each function of the targetFields returns a targetValue which is the latest updated value for the fieldDataIndex for the same targetFields object. inside the function we basically check the data from the arguments and based on certain conditions finalize the return value. in targetFields we can have fieldType listOption, fieldDataIndex as listOptions and fieldType as topOption and fieldDataIndex as one of the form field. the dependency can handle updating both types. The function can also call an api inside it. if api will be called then value of function will start with async. the function will take 10 arguments, make sure to include them even if we are not using all of them. the arguments are changedValues(latest changed field key value is present here, also note that the dependency was triggered because of editing this field and this field will be present in the ), reqdFieldValues (contains the details of all the dependentFields inside it), targetValue(current value of the field of interest for whom this dependency is calculating updated value), allValues(this is a form instance which contains data about all form fields, form list fields and table data), schema(input schema for advancedformnode), dataSource(data present for the table), moment (js library), record(some data stored in localStorage), spaceSlug, workspaceId (spaceSlug and workspaceId are related to identifying the route which is rendering the advancedformnode)
  - Null and Hardcoding check: 
    - Always add a "?" for all the variables so that no undefined is received 
    -Inside the function for a transformation dependency:
      Ensure null and undefined checks are added for every field or variable referenced (e.g., changedValues, reqdFieldValues, targetValue).
      Avoid hardcoding any values unless explicitly required in the user prompt. Always use dynamic values derived from the provided arguments.
      If looping through arrays (e.g., listOptions or tableData), verify that the array exists and has valid items (e.g., if (Array.isArray(targetValue) && targetValue.length > 0)).
      If an API call is included, validate the response and ensure error handling is implemented to avoid hardcoded fallbacks.
    -Example Logic Flow:
      Validate changedValues is not null or undefined before accessing its keys.
      Check all computed values for NaN or invalid types before returning.
      Use conditional checks to determine the return value (e.g., return updatedValue || targetValue).
  - dependentFields: An array of objects containing the fields that trigger this dependency. Each object includes:
    - fieldType: "topOption", "listOption", or "tableColumn".
    - fieldDataIndex: The name of the field triggering the dependency. For tableColumn, this key is absent.
  - targetFields: Specifies the fields to update. Each object includes:
    - fieldType: "topOption", "listOption", or "tableColumn".
    - fieldDataIndex: The name of the field being updated.
    - function: Updates the field’s value based on logic. Arguments:
      changedValues, reqdFieldValues, targetValue, allValues, schema, dataSource, moment, record, spaceSlug, workspaceId.
      - Example:
        javascript
        function(changedValues, reqdFieldValues, targetValue, allValues, schema, dataSource, moment, record, spaceSlug, workspaceId) {{ return updatedValue; }}
        

  2. Validation Dependency
  - type: "validation"
  - dependentFields: Same as transformation.
  - checkType key has value as hard/soft and it is present parallel to dependentFields and targetFields.
  - if the targetFields contains the function targetFields will also contain a message key which will be displayed for prompting the frontend.
  - description: when the validation is related to targetFields being the tableColumn dependentFields might not be present. Generally each targetFields object will contain fieldType, fieldName, ruleSchema + message or function. dependentFields object can contain fieldType and fieldDataIndex key, for the fieldType ListOption if we dont have fieldDataIndex that is fine. if the targetFields contain ruleSchema it will adhere to rules config from the ant design. When there is function you should notice that the function will return a boolean, if the function is returning true then in the frontend an error will be prompted. When we are dealing with dependency which will need function try to add only one object in dependentFields and 1 in targetFields basically keep the whole validation dependency for one field only. Add 2nd object for the other field.
  - Null and Hardcoding check: 
    - Always add a "?" for all the variables so that no undefined is received
    Inside the function for a validation dependency:
      Always check that value and currentItem exist before attempting to validate.
      Any comparisons or validations should include safeguards against undefined or unexpected values (e.g., if (value == null || isNaN(value))).
      Avoid returning hardcoded true or false values without a meaningful condition. Ensure that the logic dynamically determines the result based on input arguments.
    Example Logic Flow:
      If validating a numeric field, ensure value is parsed or cast to a number and checked for validity.
      Ensure currentItem has all the necessary properties before accessing nested fields.
  - targetFields: Same structure as transformation, with an added key:
    - check_type: "ruleSchema" or "function".
    - function: Validates input. Arguments: value, currentItem.
      - Example:
        javascript
        function(value, currentItem) {{ return value < 0; }}
        

  3. TableRowTransformation Dependency
  - type: "tableRowTransformation"
  - dependentFields: Always includes:
    - fieldType: "tableColumn".
  - whenever we are working with tableRowTransformation by default we add one dependency at the top whose code is, make sure that this is one separate dependency, do not mix this with any other dependency. Have a separate object for the other tableRowTransformation dependency: {{ type: "tableRowTransformation", targetFields: [{{ function: "function (row, reqdFieldValues) {{return row;}}"}}]}},
  - Null and Hardcoding check: 
    - Always add a "?" for all the variables so that no undefined is received
    Inside the function for a tableRowTransformation dependency:
      Verify that row exists and is a valid object before making changes.
      Use reqdFieldValues to dynamically compute any required logic, ensuring all fields are validated for null or undefined.
      Do not hardcode row properties or values; always compute based on arguments or logic.
    Example Logic Flow:
      If a row field (e.g., number1) is referenced, check it for validity (if (row.number1 != null && !isNaN(row.number1))).
      Ensure that any default values returned for the row are consistent with the schema logic.
  - targetFields: Updates rows in the table. Each object includes:
    - fieldType: "tableColumn".
    - fieldDataIndex: The name of the table column being updated.
    - function: Modifies the row. Arguments: row, reqdFieldValues, oldRow.
      - Example:
        javascript
        function(row, reqdFieldValues, oldRow) {{ row.sum = row.number1 + row.number2; return row; }}
  - function body description: tableRowTransformation works in a way that whenever user makes changes to one the field mentioned in the dependentFields the function inside the targetFields is executed and new value of the the row the is returned and these value becomes visible in the advancedformnode. If the dependency doesn’t contain dependentFields the function inside the targetFields is executed automatically. 
        

  4. TopOptionDependency
  - type: "topOptionDependency"
  - dependentFields: Always includes:
    - fieldType: "topOption".
    - fieldDataIndex: The field name triggering this dependency.
  - Null and Hardcoding check:
    - Always add a "?" for all the variables so that no undefined is received
    - Inside the function for a topOptionDependency:
        Always check that changedValues is not null or undefined before accessing its keys.
        Validate the topOptions array before mapping or modifying its elements (if (Array.isArray(topOptions) && topOptions.length > 0)).
        Do not hardcode display or any other property; dynamically determine the values based on conditions.
    - Example Logic Flow:
        Use a map function to iterate over topOptions, ensuring each element is valid before updating.
        Ensure changedValues contains the expected field and validate its value before applying any conditional logic.
  - targetFields: Updates topOptions. Each object includes:
    - fieldType: "topOption".
    - fieldDataIndex: The field being updated.
    - function: Updates visibility or values in topOptions. Arguments: changedValues, topOptions.
      - Example:
        javascript
        function(changedValues, topOptions) {{ return updatedTopOptions; }}
  - description: Whenever the user changed the field whose fieldDataIndex matches with the object inside the dependentFields the function inside the targetFields is called. the function inside targetFields contains changedValues and topOptions as the argument where topOptions is the topOptions schema provided in input to render the antd form, the changedValues will contain the changed Value of the dependentField whose fieldDataIndex has matched. What function basically do is it checks for the changedValue and based on the changed value it decides if one the other topOption field should be displayed or not. if display = ‘none’ field will not be rendered and if display = '' field will be rendered. So the conditional logic is present inside the function.
        

  5. ListOptionDependency
  - type: "listOptionDependency"
  - dependentFields: Always includes:
    - fieldType: "listOption".
    - fieldDataIndex: "listOptions".
  - Null and Hardcoding check:
    - Always add a "?" for all the variables so that no undefined is received
    - Inside the function for a listOptionDependency:
      Ensure changedValues, listOptions, and listValues are validated for null or undefined.
      Always check that listOptions is an array before applying logic to it (if (Array.isArray(listOptions))).
      Dynamically compute fields such as rows and display based on input values; avoid hardcoded configurations.
    - Example Logic Flow:
      Loop through listOptions and ensure each field referenced (e.g., rows, display) is validated and computed based on logic.
      Check listValues for the correct row index and dynamically apply updates based on the changed field’s value.
  - targetFields: Updates listOptions. Each object includes:
    - changedValues.listOptions will contain the listOptions field contained by user.
    - fieldType: "listOption".
    - fieldDataIndex: "listOptions".
    - function: Updates listOptions schema. Arguments: changedValues, listOptions, listValues.
    - function body description: Mostly this dependency is used to manage if a particular listOption field should be displayed for the particular row or not based on the user input, the keys display and rows are used for this. if display is none or the rows doesnt contain the index of the list index for the particular field then we won’t display the field for the particular list row. 2nd argument determines what fields to show for antd form list and 3rd argument contains the data for the antd form list in the advancedformnode, in the function we determine which particular row was modified by the user and then based on the value of the field which was changed we run some conditional suiting the requirement.
    - Example:lets assume currently there are 4 rows for the antd form list and now the user made change for the third row for the financial_class(assume as one of the listOption) field, so the function is checking if the financial_class field was updated or not, if it wasnt updated we are directly returning the listOptions without any change if the financial_class was changed and the value was updated to 1(expected value) then we are looping over the listOptions(input schema) when we encounter the health_plan_name(target listoption fied) field we are making sure for all the rows of the antd form list that health_plan_name is displayed by returning: return {{ ...option, display: "", rows: newRows(contains the row index where financial_class is 1) }};, if financial_class is not equal to 1 we are making sure that the antd form list rows for which were not updated and the financial_class value is 1 keep displaying the health_plan_name field and the for the rows not having financial_class as 1 dont show health_plan_name
      - Example:
        javascript
        function(changedValues, listOptions, listValues) {{ return updatedListOptions;}}
        

  6. EnableListOptionDependency
  - type: "enableListOptionDependency"
  - dependentFields: Always includes:
    - fieldType: "topOption".
    - fieldDataIndex: Field triggering this dependency.
  - Null and Hardcoding Check:
    - Always add a "?" for all the variables so that no undefined is received
    - Inside the function for an enableListOptionDependency:
      Validate changedValues before accessing any keys or applying logic.
      Return a dynamic boolean based on logic rather than hardcoded true or false.
    - Example Logic Flow:
      Ensure the field in changedValues matches the expected dependent field.
      Dynamically check the value of the dependent field and return true or false based on the condition.
  - targetFields: Enables/disables listOptions. Each object includes:
    - fieldType: "listOption".
    - function: Returns a boolean to enable/disable listOptions. Arguments: changedValues.
      - Example:
        javascript
        function(changedValues) {{ return true; }}
    - description: This dependency is used when we want to display antd form list only when the user selects a particular value for a field in the antd form. The function inside the targetFields takes in changedValues. This changedValues will be basically the change user would have made in the antd form field. This function will only be called in the antd form field which was changed by the user is present in the dependentFields. Now when the function is called the logic is executed and depending on the changed value of the antd form field a boolean is returned. if true is returned than antd form list will be rendered on the screen. 
        

  7. OnLoadDependency
  - type: "onLoadDependency"
  - description: onLoadDependency is specifically for the topOptions. This dependency is called inside the useEffect. Thus it is used to setup the updated value for the topOptions. The function will either return new top options or an object with keys newTop and newsigns the function will take in three arguments, first represents topOptions, second represents signdata third represent tableData. you should add three argument in your output irrespective of them being used. this function can check the default value of any field and do some changes such as changing display key to ‘’ (previously it would have ‘none’ thus the field was not rendered) or it can loop over table data do some process and return updated top options or it can also contain signdata and we need to return topoptions and signdata, if dependency is regarding signdata then we need to update the ref for sure using the line signatureRefs[‘signature’].current = ‘data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAZAAAADIAAAXJggg==’; here I will give the image metadata based on my requirement and that needs to be used. based on the dependencies provided below learn from them
  - Null and Hardcoding Check:
    - Always add a "?" for all the variables so that no undefined is received
    - Inside the function for an onLoadDependency:
      Validate topOptions, signData, and tabData for null or undefined before applying logic.
      When returning a combined object (e.g., {{ newTop, newsigns }}), ensure each key is computed dynamically without hardcoding values.
      If looping through tabData, validate that it is an array and has valid items.
      For signature updates, ensure the metadata value is dynamically fetched or computed and not hardcoded unless explicitly provided in the prompt.
    - Example Logic Flow:
      Check the default value of any topOptions field dynamically and update keys like display based on conditions.
      If modifying signature references, validate the reference and update it dynamically based on input arguments.
  - dependentFields: None (specific to form loading).
  - targetFields: Updates fields at load. Each object includes:
    - fieldType: "topOption".
    - fieldDataIndex: Field being updated.
    - function: Executes logic on form load. Arguments: topOptions, signData, tabData.
      - Example:
        javascript
        function(topOptions, signData, tabData) {{ return updatedSchema; }}
        

  Important Rules:
    1. Field types for topOptions and listOptions must be one of: text, number, select, date, image_capture, signature, timerange, randomuuid, random8digitbarcode.
    2. Dependency types must be one of: transformation, validation, tableRowTransformation, topOptionDependency, listOptionDependency, enableListOptionDependency, onLoadDependency.
    3. Include all required arguments in function fields to ensure proper structure and consistency.
    4. The output schema must be complete, valid, and precisely match the structure.
    5. Size values must be appropriate for form grid layout.
    6. Include all arguments for functions in targetFields to ensure proper structure and consistency.
    7. When generating schemas, include only the sections needed for the specific requirement.
    8. When generating schemas, ensure all sections adhere to these guidelines.
    9. For the functions in targetFields of the dependencies do not add any formatting or comments, simply provide the the code of the function in one single line.
    10. When generating the schema for the function make sure to add the checks for null and undefined for all the variables used inside it. additionally go through the training data and apply the kind of logic which is present in the training data.
    11. For the function body try to follow the logic method followed in the training data.

  User Requirement: {user_requirement}

  Generate a complete, valid JSON schema that fulfills these requirements while adhering to the structure and rules above.
"""

In [4]:
def get_llm_chain():
    # Read fine-tuned model name
    with open("fine_tuned_model_name.txt", "r") as f:
        fine_tuned_model = f.read().strip()

    # Setup ChatOpenAI with updated parameters
    llm = ChatOpenAI(
        model_name=fine_tuned_model,
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY"),
    )

    # Create prompt template using langchain-core
    prompt = PromptTemplate(
        template=SCHEMA_PROMPT, input_variables=["user_requirement"]
    )

    # Create chain with updated LLMChain
    chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

    return chain

In [5]:
# Call this function to train the model
# train_model()

In [6]:
chain = get_llm_chain()

In [7]:
user_requirement = """
    Provide me a schema which contains only a form (topOptions) with the following fields:
    1. First Name (required)
    2. Last Name (required)
    3. Email (required, with email validation)
    4. Phone Number (optional)
    """
result = chain.run(user_requirement=user_requirement)
print(result)

```json
{
  "topOptions": [
    {
      "id": "1",
      "name": "first_name",
      "label": "First Name",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 1,
      "size": 12
    },
    {
      "id": "2",
      "name": "last_name",
      "label": "Last Name",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 2,
      "size": 12
    },
    {
      "id": "3",
      "name": "email",
      "label": "Email",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 3,
      "size": 12
    },
    {
      "id": "4"

In [8]:
user_requirement = """
    Provide me a schema which contains the following:
    1. A form with the following topOptions fields:
        a. "First Name" (required, text field)
        b. "Last Name" (required, text field)
        c. "Email" (required, text field with email validation)
        d. "Role" (required, select field with the options: Admin, User, Manager)
        e. "Total Tasks" (read-only, number field)
    2. A Form List section with the following listOptions fields:
        a. "Task Name" (required, text field)
        b. "Assigned To" (required, select field with options fetched via sql query, by default hidden)
        c. "Due Date" (optional, date field)
        d. "Priority" (required, select field with the options: High, Medium, Low)
    3. A listOptionDependency that:
        a. Display the "Assigned To" field if the "Task Name" is not empty, otherwise keep it hidden.
    4. A transformation dependency that:
        b. Calculates and displays the total number of tasks (rows in listOptions) in a separate "Total Tasks" field in topOptions.
    """

result = chain.run(user_requirement=user_requirement)
print(result)

```json
{
  "topOptions": [
    {
      "id": "first_name",
      "name": "first_name",
      "label": "First Name",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 1,
      "size": 6
    },
    {
      "id": "last_name",
      "name": "last_name",
      "label": "Last Name",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 2,
      "size": 6
    },
    {
      "id": "email",
      "name": "email",
      "label": "Email",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 3,
      "size": 6
    },
   

In [9]:
user_requirement = """
    Provide me a schema which contains the following:

    A form with the following topOptions fields: a. "Employee ID" (required, text field)
    b. "Employee Name" (required, text field)
    c. "Department" (required, select field with options: HR, Engineering, Sales, Marketing)
    d. "Total Leaves Taken" (read-only, number field)

    A Form List section with the following listOptions fields: a. "Leave Type" (required, select field with options: Sick Leave, Casual Leave, Unpaid Leave)
    b. "Leave Start Date" (required, date field)
    c. "Leave End Date" (required, date field)
    d. "Days Taken" (read-only, number field)

    A listOptionDependency that:
    a. Dynamically calculates the "Days Taken" based on the "Leave Start Date" and "Leave End Date".

    A transformation dependency that:
    a. Sums up the "Days Taken" across all rows in listOptions and updates the "Total Leaves Taken" field in topOptions.
"""

result = chain.run(user_requirement=user_requirement)
print(result)

```json
{
  "topOptions": [
    {
      "id": "employee_id",
      "name": "employee_id",
      "label": "Employee ID",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 1,
      "size": 6
    },
    {
      "id": "employee_name",
      "name": "employee_name",
      "label": "Employee Name",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 2,
      "size": 6
    },
    {
      "id": "department",
      "name": "department",
      "label": "Department",
      "type": "select",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [
        { "display_value": "HR", "value": "HR" },
        { "display_value": "Engineerin

In [10]:
user_requirement = """
    Provide me a schema which contains the following:

    A form with the following topOptions fields: a. "Project Name" (required, text field)
    b. "Start Date" (required, date field)
    c. "End Date" (required, date field)
    d. "Budget" (required, number field)
    e. "Total Expenses" (read-only, number field)

    A Form List section with the following listOptions fields: a. "Expense Name" (required, text field)
    b. "Expense Type" (required, select field with options: Fixed, Variable)
    c. "Amount" (required, number field)

    A tableRowTransformation dependency that:
    a. Updates the "Amount" field for rows in tableData by applying a 10 percent increase if "Expense Type" is Fixed.

    A transformation dependency that:
    a. Sums up all "Amount" values in listOptions and updates the "Total Expenses" field in topOptions.

    A topOptionDependency that:
    a. Hides the "Budget" field if "Total Expenses" exceeds a certain threshold (e.g., 100,000).
"""

result = chain.run(user_requirement=user_requirement)
print(result)

```json
{
  "topOptions": [
    {
      "id": "project_name",
      "name": "project_name",
      "label": "Project Name",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 1,
      "size": 12
    },
    {
      "id": "start_date",
      "name": "start_date",
      "label": "Start Date",
      "type": "date",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 2,
      "size": 12
    },
    {
      "id": "end_date",
      "name": "end_date",
      "label": "End Date",
      "type": "date",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 3,
      

In [11]:
user_requirement = """
    Provide me a schema which contains the following:

    A form with the following topOptions fields: a. "Username" (required, text field)
    b. "Password" (required, text field with password validation)
    c. "Account Type" (required, select field with options: Free, Premium, Enterprise)
    d. "Signup Date" (read-only, date field set to today's date)

    A Form List section with the following listOptions fields: a. "Feature Name" (required, text field)
    b. "Usage Limit" (optional, number field)
    c. "Activation Date" (required, date field)
    d. "Status" (read-only, select field with options: Active, Inactive, Suspended)

    A validation dependency that:
    a. Validates that "Password" contains at least 8 characters, one uppercase letter, and one special character.
    b. Ensures that the "Usage Limit" value is greater than or equal to 0.

    An enableListOptionDependency that:
    a. Enables the listOptions section only if "Account Type" is Premium or Enterprise.
"""

result = chain.run(user_requirement=user_requirement)
print(result)

```json
{
  "topOptions": [
    {
      "id": "username",
      "name": "username",
      "label": "Username",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 1,
      "size": 12
    },
    {
      "id": "password",
      "name": "password",
      "label": "Password",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 2,
      "size": 12
    },
    {
      "id": "account_type",
      "name": "account_type",
      "label": "Account Type",
      "type": "select",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [
        { "display_value": "Free", "value": "Free" },
        { "display_value": "Premium", "value": "Pr

In [12]:
user_requirement = """
    Provide me a schema which contains the following:

    A form with the following topOptions fields: a. "Patient Name" (required, text field)
    b. "Date of Birth" (required, date field)
    c. "Gender" (required, select field with options: Male, Female, Other)
    d. "Primary Insurance" (read-only, text field fetched via API query)
    e. "Total Payments" (read-only, number field)

    A Form List section with the following listOptions fields: a. "Service Name" (required, text field)
    b. "Service Cost" (required, number field)
    c. "Discount (%)" (optional, number field)
    d. "Final Cost" (read-only, number field)

    A listOptionDependency that:
    b. Hides the "Discount (%)" field for rows where "Service Name" is "Consultation".

    A transformation dependency that:
    a. Sums up the "Final Cost" values for all rows in listOptions and updates the "Total Payments" field in topOptions. 
"""

result = chain.run(user_requirement=user_requirement)
print(result)

```json
{
  "topOptions": [
    {
      "id": "patient_name",
      "name": "patient_name",
      "label": "Patient Name",
      "type": "text",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 1,
      "size": 12
    },
    {
      "id": "date_of_birth",
      "name": "date_of_birth",
      "label": "Date of Birth",
      "type": "date",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [],
      "options_query": null,
      "onchange": null,
      "display": "",
      "order": 2,
      "size": 12
    },
    {
      "id": "gender",
      "name": "gender",
      "label": "Gender",
      "type": "select",
      "default_value": null,
      "is_read_only": false,
      "required": true,
      "options": [
        { "display_value": "Male", "value": "male" },
        { "display_value": "Female", "val